# SETUP

The first part of the code sets up the pca_wavelet network, the training comes later. Most of this code comes from the original authors

In [1]:
import sys
import time
sys.path.append('../segmentation_helper')

import tensorflow as tf
import keras
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import data_loader as dl
import model_broker as mb
import os
import pandas as pd

tf.keras.backend.set_floatx("float64")

GPU device not found
Found GPU at: 


In [2]:
def scaledtanh(x): 
    return tf.math.tanh(x*0.5)

def scaledatanh(x):
    return tf.math.atanh(x)*2.0


In [3]:
def conv_calculate_A_and_b(imghead, seghead, img_train,seg_train):
    n = 0.0
    
    imdecom_shape = imghead(next(iter(img_train))[0]).shape
    img_channels = imdecom_shape[3] # shape_0
    imdecom_2d_shape = imdecom_shape[1]*imdecom_shape[2] #shape_1
    
    segdecom_shape = seghead(next(iter(seg_train))[0]).shape
    seg_channels = segdecom_shape[3] # shape_0
    segdecom_2d_shape = segdecom_shape[1]*segdecom_shape[2] #shape_1
    

    xxt = np.zeros([img_channels,img_channels])
    yxt = np.zeros([img_channels,seg_channels])
    x = np.ones([imdecom_2d_shape])
    x_m = np.zeros([img_channels])
    y = np.ones([segdecom_2d_shape]) 
    y_m = np.zeros([seg_channels])

    bar = tqdm.notebook.tqdm(total = int(img_train.cardinality()))

    for item in iter(zip(img_train,seg_train)):
        bar.update(1)
        image = item[0][0]
        segmentation = item[1][0]

        imgdecom = imghead(image)
        segdecom = seghead(segmentation)

        mat = tf.reshape(imgdecom,[-1,seg_channels])
        segmat = tf.reshape(segdecom,[-1,img_channels])

        cov = tf.tensordot(mat,mat,[0,0])
        xxt += cov
        #del cov

        segcov = tf.tensordot(mat,segmat,[0,0])
        yxt += segcov
        #del segcov

        x_m += tf.linalg.matvec(mat,x,transpose_a=True)
        y_m += tf.linalg.matvec(segmat,y,transpose_a=True)

        n += 1
    xxt = xxt - tf.matmul([x_m],[x_m],transpose_a=True)/n
    yxt = yxt - tf.matmul([x_m],[y_m],transpose_a=True)/n
    inverse_xxt = tf.linalg.pinv(xxt)
    A = tf.matmul(inverse_xxt,yxt)
    b = (y_m - tf.linalg.matvec(A,x_m,transpose_a=True))/n
    A_filter = tf.reshape(A,(1,1,seg_channels,seg_channels))
    return A_filter,b

In [4]:
def connected_calculate_A_and_b(imghead, seghead, img_train,seg_train):
    imgflat = np.prod(imghead(next(iter(img_train))[0]).shape)
    segflat = np.prod(seghead(next(iter(seg_train))[0]).shape)
    end_shape = next(iter(seg_train))[0].shape
    n = 0.0

    xxt = np.zeros([imgflat])
    yxt = np.zeros([segflat])
    x = np.zeros([imgflat])
    y = np.zeros([segflat]) 

    bar = tqdm.notebook.tqdm(total = int(img_train.cardinality()))

    for item in iter(zip(img_train,seg_train)):

        bar.update(1)

        image = item[0][0]
        segmentation = item[1][0]

        imgdecom = imghead(image)
        segdecom = seghead(segmentation)

        mat = tf.reshape(imgdecom,[-1])
        segmat = tf.reshape(segdecom,[-1])

        cov = tf.matmul([mat],[mat],transpose_a=True)
        xxt += cov
        segcov = tf.matmul([mat],[segmat],transpose_a=True)
        yxt += segcov
        x+=mat
        y+=segmat
        n += 1
        
    print("loop calculated")
    xxt = xxt - tf.matmul([x],[x],transpose_a=True)/n
    yxt = yxt - tf.matmul([x],[y],transpose_a=True)/n
    print("calculating inverse")
    inverse_xxt = tf.linalg.pinv(xxt)
    print("calculating A")
    A = tf.linalg.matmul(inverse_xxt,yxt)
    print("calculating b")
    b = (y - tf.linalg.matvec(A,x,transpose_a=True))/n
    return A,b

In [5]:
def build_model_instance(train,
                test,
                dataset,
                model_name,
                keep_percent=1.0,
                count=3,
                sample_size=100,
                activity_regularizer=None,
                inverse_activity_regularizer=None,
                activation_before=False,
                check_build=False):
    
    stats = None
    
    broker = mb.ModelBroker(trainset=train,
                                testset=test,
                                dirname=dataset+"_"+model_name,
                                keep_percent=keep_percent,
                                count=count,
                                sample_size=sample_size,
                                activity_regularizer = activity_regularizer,
                                inverse_activity_regularizer=inverse_activity_regularizer,
                                activation_before=activation_before)
    
    head,invhead = broker.build_model()
    head,invhead = broker.load_model()    
    if check_build:
        train_psnr,train_ncc = broker.check_build(head,invhead,train,stats_only=True)
        test_psnr,test_ncc = broker.check_build(head,invhead,test,stats_only=True)
        stats = (train_psnr,train_ncc,test_psnr,test_ncc)
    return head,invhead, stats

In [6]:
def conv_metric_calculate(img_ds,seg_ds,A_filter,b):
    threshold_intensity = 0.01
    dice_coeff_vals = []
    iou_coeff_vals = []
    n = 0
    
    for image,seg_base in iter(zip(img_ds,seg_ds)):
        imgdecom = imghead(image[0])
        conv = tf.nn.conv2d(imgdecom, A_filter,1,"VALID")
        conv = tf.nn.bias_add(conv,b)
        seg = seginvhead(conv)
        y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
        y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
        dice_coeff_vals.append(dice_coef(y_true,y_pred))
        iou_coeff_vals.append(iou_coef(y_true,y_pred))
        n+=1
    return iou_coeff_vals,dice_coeff_vals,n


In [7]:
def connected_metric_calculate(img_ds,seg_ds,A,b):
    threshold_intensity = 0.01
    dice_coeff_vals = []
    iou_coeff_vals = []
    n = 0
    reconstruct = seghead(next(iter(seg_ds))[0]).shape
    for image,seg_base in iter(zip(img_ds,seg_ds)):
        imgdecom = imghead(image[0])
        imgdecom = tf.reshape(imgdecom,(1,-1))
        segdecom = tf.linalg.matvec(A,imgdecom,transpose_a=True)+b
        seg = seginvhead(tf.reshape(segdecom,(reconstruct)))
        y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
        y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
        dice_coeff_vals.append(dice_coef(y_true,y_pred))
        iou_coeff_vals.append(iou_coef(y_true,y_pred))
        n+=1
    return iou_coeff_vals,dice_coeff_vals,n

In [8]:
def calculate_metrics(seg_ds,img_ds,imghead,seghead,seginvhead,A,b,mode):
    if mode == "conv":
        iou_coeff_vals,dice_coeff_vals,n = conv_metric_calculate(img_ds,seg_ds,A,b)
    else:
        iou_coeff_vals,dice_coeff_vals,n = connected_metric_calculate(img_ds,seg_ds,A,b)
    dice_coeff_mean = sum(dice_coeff_vals)/n
    iou_coeff_mean = sum(iou_coeff_vals)/n
    dice_coeff_std = (sum([((x - dice_coeff_mean) ** 2) for x in dice_coeff_vals]) / n)**0.5
    iou_coeff_std = (sum([((x - iou_coeff_mean) ** 2) for x in iou_coeff_vals]) / n)**0.5
    return dice_coeff_mean, iou_coeff_mean, dice_coeff_std, iou_coeff_std

In [9]:
def dice_coef(y_true, y_pred,smooth=1):
    y_true_f = tf.reshape(y_true,-1)
    y_pred_f =tf.reshape(y_pred,-1)
    intersection = tf.reduce_sum(y_true_f * y_pred_f,0)

    return float((2. * intersection+smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)+smooth))

In [10]:
def iou_coef(y_true, y_pred,smooth=1):
  intersection = tf.reduce_sum(y_true * y_pred, 0)
  union = tf.reduce_sum(y_true,0)+tf.reduce_sum(y_pred,0)-intersection
  iou = tf.reduce_mean((intersection+1) / (union+1), 0)
  return float(iou)

In [33]:
counts_exp = [("count",i) for i in range(4,6)]
keep_percents_exp = [("keep_percent",i/10) for i in range(2,4)]
train_sizes_exp = [("train_size",i*2000) for i in range(1,3)] + [("train_size",None)]
res_exp = [("res",2**i) for i in range(6,9,2)]
experiments = [("nudge",0)]#+res_exp #keep_percents_exp + train_sizes_exp + counts_exp +  res_exp 

In [34]:
experiments

[('res', 64), ('res', 256)]

In [35]:

test_size = 300


#
dataset = "pets"
#change mode between conv and connected to explore differenet linear inverse methods
mode = "conv"
nudge = np.random.randint(0,1000000)

if mode == "conv":
    calculate_A_and_b = conv_calculate_A_and_b
else:
    calculate_A_and_b = connected_calculate_A_and_b

#df = pd.DataFrame()

save_name = mode+"_formal_experiment_"+str(nudge)
print(f"saving results to:{save_name}")
for settings in tqdm.notebook.tqdm(experiments):
    
    #These are the standard experiment settings
    activity_regularizer = scaledtanh
    inverse_activity_regularizer = scaledatanh
    count = 2
    keep_percent = 0.1
    train_size = 6000
    res = 128
    
    #Initialise the experiment record for this iteration
    record = pd.Series()
    
    #Change the values of the paramaters passed into the model builder and data loader
    variable, value = settings
    if variable == "count":
        count = value
    if variable == "keep_percent":
        keep_percent = value
    if variable == "train_size":
        train_size = value
    if variable == "res":
        resolution = value

    #Load the datasets
    loader = dl.DataLoader(IMAGE_SIZE=res,dataset=dataset,take=train_size)
    img_ds = loader.import_processed_img()
    seg_ds = loader.import_processed_seg()
    cardinality = int(img_ds.cardinality())

    #Generate train/test splits
    img_test = img_ds.take(test_size)
    seg_test = seg_ds.take(test_size)
    img_train = img_ds.skip(test_size)
    seg_train = seg_ds.skip(test_size)    
    
    #Record experiment parameters
    record["resolution"]=res
    record["count"]=count
    record["keep_percent"] = keep_percent
    record["activity_regularizer"] = activity_regularizer != None
    record["training_data_size"] = train_size
    
    #Training image reconstruction network
    img_train_start = time.time()
    imghead, imginvhead,stats = build_model_instance(img_train,img_test,dataset,"img",keep_percent = keep_percent,count=count,check_build=True)
    psnr_train,ncc_train,psnr_test,ncc_test = stats
    img_train_end = time.time()
    
    #Record values from image reconstruction network experiements
    record["img_channel_size"] = imghead(next(iter(img_train))[0]).shape[-1]
    record["img_train_time"] = img_train_end - img_train_start
    record["train_img_psnr"] = psnr_train
    record["train_img_ncc"] = ncc_train
    record["test_img_psnr"] = psnr_test
    record["test_img_ncc"] = ncc_test
    
    #Training segmentation reconstruction network
    seg_train_start = time.time()
    seghead, seginvhead,stats = build_model_instance(seg_train,seg_test,dataset,"seg",count=count,keep_percent = keep_percent,check_build=True)
    psnr_train,ncc_train,psnr_test,ncc_test = stats
    seg_train_end = time.time()
    
    #Record values from segmentation reconstruction network experiements
    record["seg_channel_size"] = seghead(next(iter(seg_train))[0]).shape[-1]
    record["seg_train_time"] = img_train_end - img_train_start
    record["train_seg_psnr"] = psnr_train
    record["train_seg_ncc"] = ncc_train
    record["test_seg_psnr"] = psnr_test
    record["test_seg_ncc"] = ncc_test
    
    #Calculate A and b
    train_time_start = time.time()
    A,b = calculate_A_and_b(imghead,seghead,img_train,seg_train)
    train_time_end = time.time()
    record["linear_inverse_train_time"] = train_time_end - train_time_start 
    
    #Caculate metrics of image segmentation network
    dice_mean_test, iou_mean_test, dice_std_test, iou_std_test = calculate_metrics(seg_test,img_test,imghead,seghead,seginvhead,A,b,mode)
    dice_mean_train, iou_mean_train, dice_std_train, iou_std_train = calculate_metrics(seg_train,img_train,imghead,seghead,seginvhead,A,b,mode)
    
    #Record metrics
    record["dice_mean_train"] = dice_mean_train
    record["iou_mean_train"] = iou_mean_train
    record["dice_std_train"] = dice_std_train
    record["iou_std_train"] = iou_std_train
    record["dice_mean_test"] = dice_mean_test
    record["iou_mean_test"] = iou_mean_test
    record["dice_std_test"] = dice_std_test
    record["iou_std_test"] = iou_std_test
    
    #Append experiment to dataframe, and save most recent dataframe
    df = df.append(record,ignore_index=True)
    df.to_csv(save_name)

saving results to:conv_formal_experiment_472870


  0%|          | 0/2 [00:00<?, ?it/s]

keep_percent 0.20629283704945683


<ipython-input-35-f26469611f6e>:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  record = pd.Series()


meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 5 keep_max 12.0
keep_channels 5
ufilts.shape (1, 1, 1, 27, 5)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([45 45], shape=(2,), dtype=int32)
keep_channels 9 keep_max 80.0
keep_channels 9
ufilts.shape (1, 1, 1, 45, 9)
end loop 32.0
Starting level 2
Completing 16.0
pca shape tf.Tensor([81 81], shape=(2,), dtype=int32)
keep_channels 16 keep_max 576.0
keep_channels 16
ufilts.shape (1, 1, 1, 81, 16)
end loop 16.0
saving to: models/pets_img
out.shape (1, 16, 16, 16)
keep_percent 0.20629283704945683
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 5 keep_max 12.0
keep_channels 5
ufilts.shape (1, 1, 1, 27, 5)
end loop 6

...

keep_percent 0.20629283704945683
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 5 keep_max 12.0
keep_channels 5
ufilts.shape (1, 1, 1, 27, 5)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([45 45], shape=(2,), dtype=int32)
keep_channels 9 keep_max 80.0
keep_channels 9
ufilts.shape (1, 1, 1, 45, 9)
end loop 32.0
Starting level 2
Completing 16.0
pca shape tf.Tensor([81 81], shape=(2,), dtype=int32)
keep_channels 16 keep_max 576.0
keep_channels 16
ufilts.shape (1, 1, 1, 81, 16)
end loop 16.0
saving to: models/pets_seg
out.shape (1, 16, 16, 16)
keep_percent 0.20629283704945683
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 5 keep_max 12.0
keep_channels 5
ufilts.

array([[[0.21235064, 0.13056904, 0.12527934],
        [0.18910176, 0.18442239, 0.10282461],
        [0.58046234, 0.43181059, 0.28883249],
        ...,
        [0.5070315 , 0.44938222, 0.27775353],
        [0.46484613, 0.38111356, 0.27786505],
        [0.43042949, 0.35828117, 0.2658816 ]],

       [[0.21418722, 0.21193083, 0.19700354],
        [0.2682538 , 0.24769911, 0.1634196 ],
        [0.27096236, 0.22783995, 0.13720703],
        ...,
        [0.34528691, 0.27003866, 0.2200138 ],
        [0.37828106, 0.3224923 , 0.26308548],
        [0.30720669, 0.29626921, 0.22518262]],

       [[0.29894134, 0.25201106, 0.21625786],
        [0.31395575, 0.27423024, 0.18296927],
        [0.29202834, 0.24174422, 0.10242202],
        ...,
        [0.41859683, 0.26922849, 0.25347495],
        [0.31663555, 0.2086883 , 0.21287197],
        [0.26037717, 0.25550586, 0.2095447 ]],

       ...,

       [[0.12723796, 0.14707223, 0.09262935],
        [0.47979236, 0.35389119, 0.20131238],
        [0.37564385, 0

array([[[0.32438439, 0.41873109, 0.21412234],
        [0.31707811, 0.41082811, 0.19247641],
        [0.32347175, 0.41290545, 0.20898031],
        ...,
        [0.33283955, 0.42582363, 0.21266492],
        [0.32114187, 0.41655058, 0.19571079],
        [0.31856692, 0.41268456, 0.21660611]],

       [[0.31732324, 0.41536245, 0.19465165],
        [0.3111302 , 0.41842186, 0.19850764],
        [0.27759099, 0.36478919, 0.18344273],
        ...,
        [0.32603714, 0.41544119, 0.21798407],
        [0.32039556, 0.41616118, 0.2029742 ],
        [0.32274601, 0.41650033, 0.22060356]],

       [[0.30286914, 0.40666816, 0.19674169],
        [0.32494783, 0.42187333, 0.20952819],
        [0.31556374, 0.42536765, 0.19791667],
        ...,
        [0.32542104, 0.40918902, 0.20183824],
        [0.32907811, 0.41806856, 0.21822175],
        [0.32759699, 0.41648284, 0.21541773]],

       ...,

       [[0.16698693, 0.15491872, 0.05167907],
        [0.26116467, 0.34877646, 0.158931  ],
        [0.24455231, 0

array([[[0.32438439, 0.41873109, 0.21412234],
        [0.31707811, 0.41082811, 0.19247641],
        [0.32347175, 0.41290545, 0.20898031],
        ...,
        [0.33283955, 0.42582363, 0.21266492],
        [0.32114187, 0.41655058, 0.19571079],
        [0.31856692, 0.41268456, 0.21660611]],

       [[0.31732324, 0.41536245, 0.19465165],
        [0.3111302 , 0.41842186, 0.19850764],
        [0.27759099, 0.36478919, 0.18344273],
        ...,
        [0.32603714, 0.41544119, 0.21798407],
        [0.32039556, 0.41616118, 0.2029742 ],
        [0.32274601, 0.41650033, 0.22060356]],

       [[0.30286914, 0.40666816, 0.19674169],
        [0.32494783, 0.42187333, 0.20952819],
        [0.31556374, 0.42536765, 0.19791667],
        ...,
        [0.32542104, 0.40918902, 0.20183824],
        [0.32907811, 0.41806856, 0.21822175],
        [0.32759699, 0.41648284, 0.21541773]],

       ...,

       [[0.16698693, 0.15491872, 0.05167907],
        [0.26116467, 0.34877646, 0.158931  ],
        [0.24455231, 0

  0%|          | 0/5700 [00:00<?, ?it/s]

In [45]:
peformance_df = df[["training_data_size","count","keep_percent",'dice_mean_test', 'dice_mean_train',
       'dice_std_test', 'dice_std_train',"linear_inverse_train_time",'iou_mean_test', 'iou_mean_train', 'iou_std_test', 'iou_std_train',"img_train_time","seg_train_time"]]

In [46]:
df.columns

Index(['activity_regularizer', 'count', 'dice_mean_test', 'dice_mean_train',
       'dice_std_test', 'dice_std_train', 'img_channel_size', 'img_train_time',
       'iou_mean_test', 'iou_mean_train', 'iou_std_test', 'iou_std_train',
       'keep_percent', 'linear_inverse_train_time', 'seg_channel_size',
       'seg_train_time', 'test_img_ncc', 'test_img_psnr', 'test_seg_ncc',
       'test_seg_psnr', 'train_img_ncc', 'train_img_psnr', 'train_seg_ncc',
       'train_seg_psnr', 'training_data_size'],
      dtype='object')

In [47]:
peformance_df["resolution"] = [128,128,128,128,128,128,64,256]

<ipython-input-47-a3408969bbc4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peformance_df["resolution"] = [128,128,128,128,128,128,64,256]


In [48]:
peformance_df.sort_values("dice_mean_test")

,training_data_size,count,keep_percent,dice_mean_test,dice_mean_train,dice_std_test,dice_std_train,linear_inverse_train_time,iou_mean_test,iou_mean_train,iou_std_test,iou_std_train,img_train_time,seg_train_time,resolution
3,4000.0,3.0,0.3,0.357757,0.348094,0.217754,0.221668,168.371407,0.339560,0.351278,0.177372,0.181098,242.668236,242.668236,128
2,2000.0,3.0,0.3,0.360171,0.353146,0.218342,0.220268,83.498731,0.342743,0.354416,0.177225,0.181497,124.599926,124.599926,128
0,6000.0,3.0,0.2,0.563722,0.557146,0.203462,0.204949,251.631982,0.414232,0.403709,0.193784,0.192409,297.254740,297.254740,128
1,6000.0,3.0,0.3,0.563794,0.557121,0.203485,0.204961,256.318803,0.414258,0.403662,0.193882,0.192451,355.888283,355.888283,128
6,6000.0,3.0,0.1,0.564751,0.558014,0.204359,0.205750,226.380696,0.417609,0.407084,0.195738,0.194292,327.402816,327.402816,64
7,6000.0,3.0,0.1,0.564751,0.558014,0.204359,0.205750,260.870025,0.417609,0.407084,0.195738,0.194292,327.596380,327.596380,256
4,None,3.0,0.3,0.591403,0.584397,0.196050,0.194237,328.039127,0.433932,0.424529,0.193239,0.189255,433.186814,433.186814,128
5,None,4.0,0.3,0.601955,0.595163,0.182029,0.183329,441.354905,0.447691,0.438458,0.179829,0.180376,783.940938,783.940938,128


In [ ]:
import random
reconstruct = seghead(next(iter(seg_train))[0]).shape
threshold_intensity = 0.01
skip = random.randint(0,70)
image,seg_base = next(iter(zip(img_test.skip(skip),seg_test.skip(skip))))
imgdecom = imghead(image[0])
imgdecom = tf.reshape(imgdecom,(1,-1))
segdecom = tf.linalg.matvec(A,imgdecom,transpose_a=True)+b
seg = seginvhead(tf.reshape(segdecom,(reconstruct)))
y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
plt.subplot(2,1,1)
plt.imshow(np.hstack([image[0],seg_base[0],seg[0]]))
plt.subplot(2,1,2)
plt.imshow(np.hstack([y_true,y_pred]))


In [1]:
import pandas as pd

In [71]:
pd.read_csv("conv_formal_experiment")[["dice_mean_test","dice_std_test","dice_mean_train","dice_std_train"]]

,dice_mean_test,dice_std_test,dice_mean_train,dice_std_train
0,0.516062,0.182558,0.539833,0.210776
1,0.355462,0.216621,0.348408,0.226357
2,0.345049,0.214888,0.335443,0.224915
3,0.361819,0.218507,0.354403,0.226326
4,0.349160,0.224643,0.338126,0.227734
